In [1]:
save_file1 = c3.HindcastFile.get('007e2816-3905-434b-b957-f7b0d0b03f46')
save_file2 = c3.HindcastFile.get('007e88f7-b86d-4379-a194-a73e2d003afb')

In [2]:
hycom_file = c3.HycomUtil.nc_open(save_file.file.url)
# save_file.file

NameError: name 'save_file' is not defined

In [ ]:
from datetime import datetime, timedelta
# hycom_file['time']

time_origin = datetime.strptime(hycom_file.variables['time'].__dict__['time_origin'] + ' +0000',
                                        '%Y-%m-%d %H:%M:%S %z')

In [ ]:
process(c3.HindcastFile.get('007e88f7-b86d-4379-a194-a73e2d003afb'))

In [17]:
# Prototype for prosessing hindcast files
from datetime import datetime, timedelta

def process(this):
    """ Process a single Hindcast NetCDF file into the Hindcast__Data types"""
    # extract surface data for a variable
    hycom_file = c3.HycomUtil.nc_open(this.file.url)
    time_origin = datetime.strptime(hycom_file.variables['time'].__dict__['time_origin'] + ' +0000',
                                        '%Y-%m-%d %H:%M:%S %z')
    
    # extract lat-long, or derive this from types
    # Not yet done:determine the offset for each based on the subsetOptions for this file
    # Note: for now it's just an integer list
    latitudes = range(len(hycom_file['lat']))
    longitudes = range(len(hycom_file['lon']))
    time = time_origin + timedelta(hours=hycom_file.variables['time'][:][0])

    # Create list of instantiated SurfaceHindcastData types
    # Create a parent id for each lat-long pair in the file.
    # use lat-long indicies to create a string parent id

    data_records = [
        c3.SurfaceHindcastData( # look at the code to understand, TimeDataPoint (Check out Type)
            **{
                'start': time_origin,
                'parent': 'HNDCST_SRFC_' + str(i) + '-' + str(j),
                'name': 'water_u',  # variable
                'for': time,           # timestamp
                'water_u': hycom_file.variables['water_u'][:].data[0,0,i,j],
                'water_v': hycom_file.variables['water_v'][:].data[0,0,i,j]
            }
        )
        for i in latitudes[:2]
            for j in longitudes[:2]
    ]
    # upsert to data store
    c3.SurfaceHindcastData.upsertBatch(data_records)

    # close the file ds, url
    c3.HycomUtil.nc_close(ds=hycom_file, url=this.file.url)
    
    # aggregation in space is possible but a bit more complex

In [ ]:
lat_lon_pair = c3.HycomLatLongPair(**{
                'i': 0,
                'j': 0,
                'lat': 20,  # variable
                'lon':20
            })

In [ ]:
# /**
# HycomLatLongPair.c3typ

# */
# entity type HycomLatLongPair mixes MetricEvaluatable schema name 'LT_LNG_PR' {
#     // The longitude index
#     i: integer
#     // The latitude index
#     j: integer
#     // The latitude value
#     lat: double
#     // The longitude value
#     lon: double
#     // The LatLong pair values
#     pair: LatLong
#     // The collection of {@link SurfaceHindcastDataSeries} records
#     surfaceHindcastData: [SurfaceHindcastDataSeries](latLongPair)
#     //
#     //subsurfaceHindcasatData: [SubsurfaceHindcastDataSeries]

# }

In [ ]:
data_records = [
        c3.SurfaceHindcastDataSeries(
            **{
                'latLongPair': lat_lon_pair,
            }
        )
    ]

In [ ]:
# Define simple metric
my_metric = c3.SimpleMetric(id = "AverageWaterU_HycomLatLongPair",
                            name = "AverageWaterU",
                            description = "Calculates average surface X-velocity for the given interval",
                            srcType = "HycomLatLongPair",
                            path = "surfaceHindcastData",
                            expression = "avg(avg(normalized.data.water_u))"
                           )
            
my_metric.toJson()
# Create EvalMetricsSpec
my_spec = c3.EvalMetricsSpec(
            ids = ["GOMu0.04_0-0"],
            expressions = ["AverageWaterU"], # does this refer to the one above?
            start = "2021-09-01",
            end = "2021-09-30",
            interval = "HOUR"
        )

In [ ]:
evalMetricsResult = c3.HycomLatLongPair.evalMetricsWithMetadata(spec=my_spec,
                                                      overrideMetrics=[my_metric])
evalMetricsResult

In [ ]:
# convert EvalMetricsResult to Pandas DataFrame
df = c3.EvalMetricsResult.toPandas(result=evalMetricsResult)
df